In [1]:
import pandas as pd

In [7]:
m=pd.read_csv("csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
m

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,41975,42033,42092,42297,42463,42609,42795,42969,43035,43240
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,23210,23705,24206,24731,25294,25801,26211,26701,27233,27830
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,60800,61381,62051,62693,63446,64257,65108,65975,66819,67679
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,5135,5319,5383,5437,5477,5567,5616,5725,5725,5872
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,12223,12335,12433,12680,12816,12953,13053,13228,13374,13451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,57226,57657,58158,58838,59422,60065,60784,61514,62167,63031
265,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10
266,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,2067,2070,2070,2071,2071,2071,2071,2072,2072,2072
267,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,16819,16908,16954,16971,16997,17036,17056,17093,17097,17123


In [127]:
def getSeriesByCountry(m,n_country,rus_country):
    t=pd.DataFrame(list(zip(m.columns[4:],list(m.iterrows())[n_country][1][4:])))
    t.columns=("Дата",rus_country)
    t=t.set_index("Дата")
    return t


In [130]:
t=getSeriesByCountry(m,209,"Россия")
t["США"]=getSeriesByCountry(m,244,"США")
t["Германия"]=getSeriesByCountry(m,130,"Германия")
t["Великобритания"]=getSeriesByCountry(m,258,"Великобритания")
t

,Россия,США,Германия,Великобритания
Дата,,,,
1/22/20,0,1,0,0
1/23/20,0,1,0,0
1/24/20,0,2,0,0
1/25/20,0,2,0,0
1/26/20,0,5,0,0
...,...,...,...,...
11/11/20,1822345,10399325,738094,1256725
11/12/20,1843678,10560111,762832,1290195
11/13/20,1865395,10737335,785093,1317496


In [37]:
list(m.iterrows())[209][1][4:]

1/22/20           0
1/23/20           0
1/24/20           0
1/25/20           0
1/26/20           0
             ...   
11/11/20    1822345
11/12/20    1843678
11/13/20    1865395
11/14/20    1887836
11/15/20    1910149
Name: 209, Length: 299, dtype: object

In [57]:
t=pd.DataFrame(list(zip(m.columns[4:],list(m.iterrows())[209][1][4:])))
t.columns=("Дата","Россия")
t.set_index("Дата")

,Россия
Дата,
1/22/20,0
1/23/20,0
1/24/20,0
1/25/20,0
1/26/20,0
...,...
11/11/20,1822345
11/12/20,1843678
11/13/20,1865395


In [41]:
zip

zip

In [125]:
m["Country/Region"][130]

'Germany'

In [106]:
str(m["Province/State"][209])

''

In [96]:
list(m.iterrows())[209][1][0]

''